# TODO: IMPROVE

In [18]:
# Import internal libraries
import json
from copy import deepcopy

# Import custom libraries
from helper import intersection, factors

# Import external libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [19]:
# SET sweep data
setdata = pd.read_csv(f"../data/sweep/setsweep33_newest2.csv", delimiter="\t", names=["addr", "t", "vwl", "vbl", "pw"] + [f"gi[{i}]" for i in range(48)] + [f"gf[{i}]" for i in range(48)])
setdata = pd.concat([setdata[["addr","t","vwl","vbl","pw",f"gi[{i}]",f"gf[{i}]"]].rename(columns={f"gi[{i}]" : "gi", f"gf[{i}]" : "gf"}) for i in range(48)])
setdata = setdata[setdata["gf"] != 0]
setdata

,addr,t,vwl,vbl,pw,gi,gf
0,46141,1.698431e+09,0,0,1,8,8.0
1,11735,1.698431e+09,0,1,1,7,7.0
2,41965,1.698431e+09,0,2,1,9,9.0
3,24212,1.698431e+09,0,3,1,7,7.0
4,63363,1.698431e+09,0,4,1,8,9.0
...,...,...,...,...,...,...,...
32896,48805,1.698432e+09,0,0,1,12,12.0
32897,33704,1.698432e+09,0,1,1,9,9.0
32898,62039,1.698432e+09,0,2,1,8,8.0
32899,31298,1.698432e+09,0,3,1,7,6.0


In [20]:
# FIRST-ORDER OPTIMIZATION
p = 0.999
# Iterate over number of bits per cell
for bpc in range(1, 3):
  # Load configuration template
  bpc_config = json.load(open(f"../settings/{bpc}bpc_best_manual.json"))
  for i, level_settings in enumerate(bpc_config["level_settings"]):
    # Get levels
    write_between = level_settings["adc_lower_write_ref_lvl"], level_settings["adc_upper_write_ref_lvl"]
    read_lvl = level_settings["adc_upper_read_ref_lvl"]

    # Generate SET experiment configs
    configs = []
    if (i != 0) and (i != (2**bpc-1)): # No SET optimization needed for lowest level
      # Get data
      d = setdata[(setdata["gi"].isin(range(write_between[0]-1))) & (setdata["pw"] == 1) & (setdata["vwl"] == 0)][["vbl","gf"]]
      d = setdata[(setdata["gi"].isin(range(20))) & (setdata["pw"] == 1) & (setdata["vwl"] == 0)][["vbl","gf"]]

      # Plot sweep
      pw = 1
      plt.figure(figsize=(5,4))
      plt.scatter(d['vbl'], d['gf'], marker='.', alpha=0.002)
      plt.plot(d.groupby('vbl').quantile(p), label=f"Max final $g$ ({p*100}$^{{th}}$ perc.)")
      plt.plot(d.groupby('vbl').mean(), label=f"Avg final $g$ ({p*100}$^{{th}}$ perc.)")
      # plt.plot(d.groupby('vbl').quantile(1-p), label="Min final $g$")
      plt.axhline(write_between[0], color='r')
      plt.axhline((write_between[1] + write_between[0])/2, color='r', linestyle='--')
      plt.axhline(write_between[1], color='r')
      
      # Get tail-start, avg-opt, tail-stop
      tailstart = intersection(np.array([0,31]),np.array([write_between[1], write_between[1]]), d.groupby('vbl').quantile(p).reset_index()['vbl'], d.groupby('vbl').quantile(p).reset_index()['gf'])
      avgopt = intersection(np.array([0,31]),np.array([(write_between[1] + write_between[0])/2, (write_between[1] + write_between[0])/2]), d.groupby('vbl').mean().reset_index()['vbl'], d.groupby('vbl').mean().reset_index()['gf'])
      tailstop = intersection(np.array([0,31]),np.array([write_between[0], write_between[0]]), d.groupby('vbl').quantile(1-p).reset_index()['vbl'], d.groupby('vbl').quantile(1-p).reset_index()['gf'])
      plt.axvline(tailstart[0][-1], color='purple', linestyle='--', label="Tail-optimized $V_{BL}$")
      plt.axvline(avgopt[0][-1], color='tan', linestyle='--', label="Mean-optimized $V_{BL}$")
      # plt.axvline(tailstop[0][-1], color='pink', linestyle='--', label="Tail-optimized $V_{bl}$ stop")
      plt.plot(tailstart[0][-1], tailstart[1][-1], '*', color='purple', markersize=12)
      plt.plot(avgopt[0][-1], avgopt[1][-1], '*', color='tan', markersize=12)
      # plt.plot(tailstop[0][-1], tailstop[1][-1], '*', color='pink', markersize=12)
      tailstart, avgopt= int(np.round(tailstart[0][0])), int(np.round(avgopt[0][0]))
      try:
        tailstop = int(np.round(tailstop[0][0]))
      except IndexError:
        tailstop = 31
      print("tailstart, avgopt, tailstop =", tailstart, avgopt, tailstop)

      # Final plot stuff
      plt.title(f"SET Sweep ($V_{{DDLS}}$ = 3.3V, $V_{{WL}}$ DAC = 0, $t_{{pw}}$ = {pw*10}ns)", y=1)
      plt.xlabel("BL DAC Level")
      plt.ylabel("Final ADC Level")
      plt.ylim(-1, 65)
      plt.legend(fontsize=9)
      plt.tight_layout()
      plt.savefig(f"../analysis/figs/{bpc}bpc_lvl{i}_sweep_opt.pdf")
      plt.show()

      # Generate configurations
      stop = 31
      j = 0 # experiment number
      for pw in range(1, 2):
        for start in range(tailstart, avgopt + 1):
          for step in sorted(list(factors(stop - start)))[:4]:
            # Copy config template from form.json
            config = deepcopy(bpc_config)

            # Update level settings
            config["level_settings"][i].update({"pw_set_start": pw, "pw_set_stop": pw + 1, "bl_dac_set_lvl_start": start, "bl_dac_set_lvl_stop": stop, "bl_dac_set_lvl_step": step})

            # Write out experiments as configs
            with open(f"configs/config_{bpc}bpc_lvl{i}_{str(j).zfill(4)}.json", 'w') as outf:
              json.dump(config, outf, indent=1)

            # Increment j
            j += 1


tailstart, avgopt, tailstop = 3 13 26


/tmp/ipykernel_1779624/3742304437.py:53: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
